- Takes as input the (768, 768, 64) images from `dataset-1`.
- It takes 4^3 average across the 2D, and average across the depth
- This results in a (192, 192, 16) volume.
- I have a image that is of shape (1, 192, 192, 16). Divide the image and the labels into 32 patches: (1, 8*24, 8*24, 16) → (24, 24, 1024) → (576, 1024). 
- Pass the image into a transformer architecture with 6 layers, which will return (576, 1) tensor, which corresponds to probabilities of pixel occuring. You should also pass the “positional embeddings”, which should be of dimension 1024.

# 4^3 3D average 


In [1]:
import numpy as np
import torch
import torch.nn as nn

def avg_3d(volume):
    # Convert the numpy array to a PyTorch tensor
    volume_tensor = torch.tensor(volume, dtype=torch.float32)

    # Add batch and channel dimensions to the tensor
    volume_tensor = volume_tensor.permute(0, 3, 1, 2)  # Reorder dimensions to (batch, channels, height, width)

    # Create the 3D average pooling layer with the appropriate kernel size and stride values
    avg_pool = nn.AvgPool3d(kernel_size=4, stride=4, padding=0)

    # Apply the average pooling layer to the input tensor
    with torch.no_grad():
        filtered_volume_tensor = avg_pool(volume_tensor)

    # Convert the output tensor back to a numpy array
    filtered_volume = filtered_volume_tensor.permute(0, 2, 3, 1)  # Reorder dimensions back to (batch, height, width, channels)
    
    return filtered_volume


# Example volume
volume = np.random.rand(1, 768, 768, 64)
volume_avgd = avg_3d(volume)
# %time avg_3d(volume) # 85 ms!

In [2]:
volume_avgd.shape

torch.Size([1, 192, 192, 16])

# Take random 3 channels

In [3]:
volume_avgd_new = volume_avgd.permute(3, 1, 2, 0)
# take 3 random numbers between 0 and 16, without replacement
random_numbers = np.random.choice(16, 3, replace=False)
volume_avgd_new = volume_avgd_new[random_numbers, :, :, :]
# Repermute
volume_avgd_new = volume_avgd_new.permute(3, 1, 2, 0)
volume_avgd_new.shape


torch.Size([1, 192, 192, 3])

# Reshape

In [4]:
import numpy as np


def reshape_img(image):
    # Calculate the size of each patch
    patch_size = 24

    B, H, W, C = image.shape
    image = image.reshape(B, H // patch_size, patch_size, W // patch_size, patch_size, C) # (B, 8, 24, 8, 24, 16)
    image = image.permute(0, 2, 4, 1, 3, 5) # (B, 24, 24, 8, 8, 16)
    image = image.reshape(B, patch_size, patch_size, -1) # (B, 24, 24, 1024)
    image = image.reshape(B, -1, 1024) # (B, 576, 1024)
    
    return image

# Example image
image = np.random.rand(2, 192, 192, 16)
image = torch.tensor(image, dtype=torch.float32)


image = reshape_img(image)

print(image.shape)


torch.Size([2, 576, 1024])


# Metric competition


In [5]:
def f0point5_score(output, target):
    # Flatten the output and target tensors
    output = output.view(-1)
    target = target.view(-1)
    
    # Convert the output to binary values 0 and 1
    output = (output > 0.5).float()
    
    # Calculate the precision and recall
    tp = torch.sum(output * target)
    fp = torch.sum(output * (1 - target))
    fn = torch.sum((1 - output) * target)
    precision = tp / (tp + fp + 1e-7)
    recall = tp / (tp + fn + 1e-7)
    
    # Calculate the F0.5 score
    beta = 0.5
    f0point5 = (1 + beta**2) * precision * recall / (beta**2 * precision + recall + 1e-7)
    return f0point5

# Model: ResNet50Seg

In [6]:
import torch.nn as nn
import torchvision.models as models

class ResNet50Seg(nn.Module):
    def __init__(self, in_channels=3, out_channels=1, class_one_weight=1):
        super(ResNet50Seg, self).__init__()
        
        self.class_one_weight = class_one_weight
        
        # Load the pre-trained ResNet50 model
        self.resnet50 = models.resnet50()
        
        # Replace the final layer to output 1024 channel instead of 1000
        self.resnet50.fc = nn.Linear(2048, 1024)
        
        
    def forward(self, x, targets=None):
        x = self.resnet50(x)
        # Apply sigmoid
        x = torch.sigmoid(x)
        
        out = {}
        
        if targets is None:
            loss = None
            precision = None
            accuracy = None
            f0point5 = None
        else:
            # Calculate the loss
            
            # Calculate class weights based on the imbalance
            class_weights = torch.tensor([1.0, self.class_one_weight]) # weight 1 for class 0, weight 5 for class 1

            # Instantiate the loss function
            loss_function = nn.BCEWithLogitsLoss(pos_weight=class_weights[1], reduction='mean')
            
            # Flatten the targets tensor
            targets = targets.reshape(-1, 1024)
            
            # Calculate the loss
            loss = loss_function(x, targets)
            
            # Calculate the accuracy: number of correctly predicted pixel / total number of pixels
            # Convert the predictions to binary values 0 and 1
            predictions = (x > 0.5).float()
            # Calculate the accuracy
            accuracy = (predictions == targets).float().mean()
            
            # Calculate the precision
            tp = torch.sum(predictions * targets)   
            fp = torch.sum(predictions * (1 - targets))
            fn = torch.sum((1 - predictions) * targets)
            precision = tp / (tp + fp + 1e-7)
            
            # Calculate F0.5 score
            f0point5 = f0point5_score(predictions, targets) 
        
        out = {
            "loss": loss,
            "logits": x,
            "accuracy": accuracy,
            "precision": precision,
            "f0point5": f0point5
        }
        
        return out


In [7]:
model = ResNet50Seg(in_channels=3, out_channels=1)
x = torch.randn(1, 3, 192, 192)  # input image
output = model(x)
output['logits'].shape

torch.Size([1, 1024])

In [8]:
# number of 1e6 parameters
sum(p.numel() for p in model.parameters()) / 1e6

25.606208

# Loss function

In [9]:
# Calculate class weights based on the imbalance
class_weights = torch.tensor([1.0, 5.0]) # weight 1 for class 0, weight 5 for class 1

# Instantiate the loss function
loss_function = nn.BCEWithLogitsLoss(pos_weight=class_weights[1], reduction='none')

# Example output and targets
out = [[0.1], [0.2], [0.9], [0.9], [0]]
tar = [[1], [1], [0], [1], [0]]
output = torch.tensor([out], dtype=torch.float32)
targets = torch.tensor([tar], dtype=torch.float32)
loss_function(output, targets)

tensor([[[3.2220],
         [2.9907],
         [1.2412],
         [1.7058],
         [0.6931]]])

In [10]:
# Calculate class weights based on the imbalance
class_weights = torch.tensor([1.0, 5.0]) # weight 1 for class 0, weight 5 for class 1

# Instantiate the loss function
loss_function = nn.BCEWithLogitsLoss(pos_weight=class_weights[1], reduction='mean')

# Example output and targets
out = [[0.1], [0.2], [0.9], [0.9], [0]]
tar = [[1], [1], [0], [1], [0]]
output = torch.tensor([out], dtype=torch.float32)
targets = torch.tensor([tar], dtype=torch.float32)
loss_function(output, targets)

tensor(1.9705)

# Model with loss test

In [11]:
model = ResNet50Seg(in_channels=3, out_channels=1)
x = torch.randn(16, 3, 192, 192)  # input image
targets = torch.randn(16, 1024)  # input image

# Normalize x to be between 0 and 1
x = torch.sigmoid(x)

# Clip targets to 0 and 1
targets = torch.clamp(targets, 0, 1)


output = model(x, targets)
output['logits'].shape, output['loss'], output['accuracy']

(torch.Size([16, 1024]),
 tensor(0.8201, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>),
 tensor(0.3237))

In [12]:
# Expected value for the BCE loss if there is 1024 elements

# Instantiate the loss function
loss_function = nn.BCEWithLogitsLoss(pos_weight=class_weights[1], reduction='mean')

y_true = torch.randn(1024, 1024) 
y_pred = torch.randn(1024, 1024) 

loss = loss_function(y_pred, y_true)
loss

tensor(0.8094)

In [13]:
# Expected value for the BCE loss if there is 1024 elements that were all correctly classified

# Instantiate the loss function
loss_function = nn.BCEWithLogitsLoss(pos_weight=class_weights[1], reduction='mean')

y_pred = torch.randn(1024*16, 1024) 

loss = loss_function(y_pred, y_pred)
loss

tensor(-2.1933)

# Overfit on 1 batch of random numbers

In [14]:
from tqdm import tqdm

In [15]:


model = ResNet50Seg(in_channels=3, out_channels=1, class_one_weight=5)
x = torch.randn(4, 3, 192, 192)  # input image
# targets = random integers of shape (4, 1024) between 0 and 1
targets = torch.randint(0, 2, (4, 1024)).float()
# Make 80% of the targets to be 0
targets[:, 0:819] = 0


# Normalize x to be between 0 and 1
x = torch.sigmoid(x)

# Clip targets to 0 and 1
targets = torch.clamp(targets, 0, 1)

# optimizer 
# optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)

n_iters = 450
for i in tqdm(range(n_iters)):
    
    # Forward pass
    output = model(x, targets)
    loss = output['loss']
    precision = output['precision']
    accuracy = output['accuracy']
    f0point5 = output['f0point5']
    
    # Backward pass
    loss.backward()
    
    # Update weights
    optimizer.step()
    
    if i % 10 == 0:
        print(f"Loss: {loss.item():.4f}, Accuracy: {accuracy.item():.4f}, Precision: {precision.item():.4f}, F0.5: {f0point5.item():.4f}")


  0%|          | 1/450 [00:00<03:06,  2.41it/s]

Loss: 1.1121, Accuracy: 0.4919, Precision: 0.0982, F0.5: 0.1171


  2%|▏         | 11/450 [00:03<02:16,  3.22it/s]

Loss: 1.0670, Accuracy: 0.6594, Precision: 0.2008, F0.5: 0.2367


  5%|▍         | 21/450 [00:06<02:12,  3.25it/s]

Loss: 1.0315, Accuracy: 0.7725, Precision: 0.2935, F0.5: 0.3401


  7%|▋         | 30/450 [00:09<02:19,  3.01it/s]


KeyboardInterrupt: 

# Data Loader

In [22]:
from torch.nn.functional import avg_pool2d
from torch.utils.data import Dataset, DataLoader
import os


class ImageSegmentationDataset(Dataset):
    def __init__(self, root, mode='train', device='cpu', cache_refresh_interval=None, cache_n_images=64):
        
        assert mode in ['train', 'test'], "mode must be either 'train' or 'test'"
        
        self.root = root
        self.mode = mode
        self.cache_refresh_interval = cache_refresh_interval
        self.cache_n_images = cache_n_images
        self._load_data()
        
    def __len__(self):
        return len(self.volume_images)
    
    # def __getitem__(self, item):
        
    #     volume = np.load(self.volume_images[item])
    #     label = np.load(self.label_images[item])
        
    #     # convert to torch tensors
    #     volume = torch.tensor(volume, dtype=torch.float32)
    #     label = torch.tensor(label, dtype=torch.float32)
        
    #     # send to device
    #     volume = volume.to(self.device)
    #     label = label.to(self.device)
        
    #     # unsqueeze the channel dimension
    #     label = label.unsqueeze(-1)
        
        
    #     # take one random number from 0 to 20, 20 to 35, 35 to 50, 50 to 64
    #     idx_1 = np.random.randint(0, 20)
    #     idx_2 = np.random.randint(20, 35)
    #     idx_3 = np.random.randint(35, 50)
    #     indices = [idx_1, idx_2, idx_3]
    #     volume = volume[:, :, indices]
        
    #     # Apply 2D average pooling to the volume and label tensors
    #     # Average each channel separately
    #     volume = volume.permute(2, 0, 1)  # Permute dimensions for avg_pool2d
    #     volume = avg_pool2d(volume, kernel_size=4, stride=4)
    #     volume = volume.permute(1, 2, 0)  # Permute dimensions back
        
    #     label = label.permute(2, 0, 1)  # Permute dimensions for avg_pool2d
    #     label = avg_pool2d(label, kernel_size=4, stride=4)
    #     label = label.permute(1, 2, 0)  # Permute dimensions back
        
    #     return {
    #         "image": volume,
    #         "targets": label
    #     }
       
    def _load_data(self):
        # Get the volume paths
        self.volume_images = os.listdir(os.path.join(self.root, self.mode, 'volume'))
        self.volume_images = [os.path.join(self.root, self.mode, 'volume', image) for image in self.volume_images]
        
        # Get label paths
        self.label_images = os.listdir(os.path.join(self.root, self.mode, 'label'))
        self.label_images = [os.path.join(self.root, self.mode, 'label', image) for image in self.label_images]
        
        # take self.cache_n_images random images from the dataset. They cannot be repeated
        if self.cache_n_images is not None and self.cache_n_images < len(self.volume_images):
            self.volume_images = np.random.choice(self.volume_images, size=self.cache_n_images, replace=False)
            self.label_images = np.random.choice(self.label_images, size=self.cache_n_images, replace=False)
        
        # Load the data into memory
        self.cached_data = []
        for volume_image, label_image in zip(self.volume_images, self.label_images):
            volume = np.load(volume_image)
            label = np.load(label_image)
            
            # convert to torch tensors
            volume = torch.tensor(volume, dtype=torch.float32)
            label = torch.tensor(label, dtype=torch.float32)
            
            self.cached_data.append({
                "image": volume,
                "targets": label
            }) 
    
    def __len__(self):
        return len(self.cached_data)
    
    def __getitem__(self, item):
        return self.cached_data[item]
    
    def refresh_cache(self):
        self._load_data()

In [23]:

# Define the dataset and dataloader
root = "../../datasets/dataset-1/"

# Mac m1 device
device = torch.device("mps")

batch_size = 16

dataset = ImageSegmentationDataset(root=root, mode='train', device=device, cache_refresh_interval=160, cache_n_images=batch_size)

In [24]:
# dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=4, pin_memory=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [25]:
# one batch
batch = next(iter(dataloader))
batch['image'].shape, batch['targets'].shape

(torch.Size([5, 192, 192, 3]), torch.Size([5, 192, 192, 1]))

In [26]:
# Let's check how fast the dataloader is

import time

start = time.time()
for i in range(100):
    batch = next(iter(dataloader))
    # Refresh the cache 
    if i % dataset.cache_refresh_interval == 0:
        dataset.refresh_cache()
    
end = time.time()

print(f"Time taken for 100 iterations: {end - start:.4f} seconds")

Time taken for 100 iterations: 0.0349 seconds


In [ ]:
%timeit np.load("/Users/viktorcikojevic/Insync/cikojevic.viktor@gmail.com/Google Drive/Kaggle/vesuvius-challenge/datasets/dataset-1/train/volume/0.npy")

49.1 ms ± 3.41 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
x = np.load("/Users/viktorcikojevic/Insync/cikojevic.viktor@gmail.com/Google Drive/Kaggle/vesuvius-challenge/datasets/dataset-1/train/volume/0.npy")
print(x.shape)
x = x[::4, ::4, :3]
print(x.shape)
np.save("/Users/viktorcikojevic/Insync/cikojevic.viktor@gmail.com/Google Drive/Kaggle/vesuvius-challenge/datasets/dataset-1/train/tmp/x.npy", x)

(768, 768, 64)
(192, 192, 3)


In [ ]:
%timeit np.load("/Users/viktorcikojevic/Insync/cikojevic.viktor@gmail.com/Google Drive/Kaggle/vesuvius-challenge/datasets/dataset-1/train/tmp/x.npy")

132 µs ± 2.63 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [ ]:
%timeit np.load("/Users/viktorcikojevic/Insync/cikojevic.viktor@gmail.com/Google Drive/Kaggle/vesuvius-challenge/datasets/dataset-1/train/label/0.npy")

122 µs ± 14.2 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Conclusion: make it's best to generate dataset-1 by taking 4x4 avg_2d while creating the dataset. This will make the dataloader much much faster.

# 1